In [1]:
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as func
from PIL import Image
from datasetCreator import ImageSubset
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from models import VGG

In [27]:
test = pd.read_csv('data/BNPP_DT_test_with_ages.csv')
val = pd.read_csv('data/BNPP_DT_val_with_ages.csv')
train = pd.read_csv('data/BNPP_DT_train_with_ages.csv')
f = h5py.File('data/bnpp_frontalonly_1024_10.hdf5')

In [28]:
train

,unique_key,bnpp_value_log,BNPP_weight,PNA_mask,PNA_wight_mask,BNP_value,age_at_sampletime
0,Flodure_51396672_img1,0.690196,1,None,1,4.9,45
1,Okepik_50995691_img1,0.690196,1,None,1,4.9,53
2,Kuesluco_51713925_img1,0.690196,1,None,1,4.9,54
3,Gumepra_51602745_img1,0.690196,1,None,1,4.9,58
4,Gapeklik_51060218_img1,0.690196,1,None,1,4.9,19
...,...,...,...,...,...,...,...
21369,Puesrueci_53095611_img1,4.845099,1,None,1,70000.1,57
21370,Adakas_52269723_img1,4.845099,1,None,1,70000.1,73
21371,Ekugud_52967627_img1,4.845099,1,None,1,70000.1,53
21372,Beguchouf_52038583_img1,4.845099,1,None,1,70000.1,60


In [50]:
train.iloc[6,:]

unique_key           Nisrana_50554206_img1
bnpp_value_log                    0.690196
BNPP_weight                              1
PNA_mask                              None
PNA_wight_mask                           1
BNP_value                              4.9
age_at_sampletime                       49
Name: 6, dtype: object

In [51]:
intersectionKeys = []
imageKeys = [n for n in f.keys()]

In [52]:
for i in train['unique_key']:
    if i in imageKeys:
        intersectionKeys.append(i)

In [54]:
len(intersectionKeys)

562

In [55]:
initialTraining = ['Umefet_51429244_img1',
'Umefos_52017419_img1',
'Umefuk_52167701_img1',
'Umegtueb_50757407_img1',
'Umeley_52554055_img1',
'Umelim_50697149_img1',
'Umellum_52452344_img1']

In [58]:
for i in initialTraining:
    valid = i in intersectionKeys
    print(f"{i} in intersection: {valid}")

Umefet_51429244_img1 in intersection: True
Umefos_52017419_img1 in intersection: True
Umefuk_52167701_img1 in intersection: True
Umegtueb_50757407_img1 in intersection: True
Umeley_52554055_img1 in intersection: True
Umelim_50697149_img1 in intersection: True
Umellum_52452344_img1 in intersection: False


In [4]:
#list(f.keys())
data = {}
ind=0
for key in list(f.keys()):
    image = f[key][:]
    #resizing to 224 x 224
    formatted = (image * 255 / np.max(image)).astype('uint8')
    img = Image.fromarray(formatted).resize((224,224))
    data[ind]=[key,img]
    #img_tensor = transform(img)
    #data[key] = img_tensor
    img.save(f'data/images/{key}.jpg')
    ind+=1

In [5]:
transform = transforms.Compose([
    transforms.PILToTensor(),
    transforms.Normalize(mean=(0.5), std=(0.5))
])
train_data = ImageSubset(csv_file = 'data/BNPP_DT_train_with_ages.csv', data=data, img_dir='data/images/')
train_data

In [6]:
train[train['unique_key']=='Umellum_52452344_img1']
#train_data.__getitem__(6)

,unique_key,bnpp_value_log,BNPP_weight,PNA_mask,PNA_wight_mask,BNP_value,age_at_sampletime


In [7]:
train_dataloader = DataLoader(train_data,batch_size=16)
train_dataloader

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = VGG('VGG16').to(device)
model

VGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [9]:
x = torch.randn(2, 1, 224, 224)

In [10]:
out = model(x)
out

/opt/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


tensor([[0.3262],
        [0.2428]], grad_fn=<AddmmBackward0>)

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"[INFO]: Computation device: {device}")
epochs = 10
batch_size = 32

[INFO]: Computation device: cpu


In [12]:
total_params = sum(p.numel() for p in model.parameters())
print(f"[INFO]: {total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"[INFO]: {total_trainable_params:,} trainable parameters.")
# the loss function
criterion = nn.L1Loss()
# the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0005)

[INFO]: 134,271,937 total parameters.
[INFO]: 134,271,937 trainable parameters.


In [13]:
# training
def train(model, trainloader, optimizer, criterion):
    model.train()
    print('Training')
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    for i, data in tqdm(enumerate(trainloader), total=len(trainloader)):
        counter += 1

        image, bnpp = data[0], data[2]
        #print(i)
        #print(data)
        #print(data[0])
        #print(data[1])
        #print(data[2])
        #break
        image = image.to(device)
        bnpp = bnpp.to(device)

        optimizer.zero_grad()
        # forward pass
        outputs = model(image.float())
        # calculate the loss
        loss = criterion(outputs, bnpp)
        train_running_loss += loss.item()
        # calculate the accuracy
        #_, preds = torch.max(outputs.data, 1)
        #train_running_correct += (preds == labels).sum().item()
        loss.backward()
        optimizer.step()

    epoch_loss = train_running_loss / counter
    #epoch_acc = 100. * (train_running_correct / len(trainloader.dataset))
    return epoch_loss, epoch_acc

In [14]:
train(model, train_dataloader,optimizer,criterion)

Training


  0%|                                                    | 0/47 [00:00<?, ?it/s]


tensor([[[ 13,  10,   7,  ...,   6,   6,   6],
         [ 13,  10,   7,  ...,   6,   6,   6],
         [ 13,  10,   7,  ...,   6,   6,   6],
         ...,
         [ 64,  53,  41,  ...,  24,  25,  25],
         [107,  93,  77,  ...,  59,  60,  61],
         [178, 163, 146,  ..., 133, 134, 135]]], dtype=torch.uint8)
Umefet_51429244_img1
3.669409867287782
tensor([[[ 49,  49,  48,  ..., 201, 202, 203],
         [ 49,  49,  48,  ..., 200, 202, 203],
         [ 49,  49,  49,  ..., 200, 201, 203],
         ...,
         [ 52,  53,  53,  ..., 160, 171, 172],
         [ 52,  53,  53,  ..., 159, 170, 171],
         [ 52,  53,  53,  ..., 158, 169, 170]]], dtype=torch.uint8)
Umefos_52017419_img1
3.4465371670736435
tensor([[[ 75,  75,  74,  ..., 124, 131, 180],
         [ 59,  58,  58,  ...,  91,  98, 147],
         [ 44,  44,  44,  ...,  60,  66, 115],
         ...,
         [ 77,  84,  92,  ...,  47,  51, 106],
         [ 87,  94, 102,  ...,  62,  62, 109],
         [125, 132, 141,  ..., 106, 10

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
# our transforms will differ a bit from the VGG paper
# as we are using the MNIST dataset, so, we will directly resize...
# ... the images to 224x224 and not crop them and we will not use...
# ... any random flippings also
train_transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize(mean=(0.5), std=(0.5))])
valid_transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize(mean=(0.5), std=(0.5))])